=============================================================================
Title: OpenAI Agents Streaming Example with function_tool
Created: 13, Jun, 2025
Author: Ted, Jung 
Description:
   A streaming event from an agent
   https://openai.github.io/openai-agents-python/ref/stream_events/#agents.stream_events.RunItemStreamEvent.item
=============================================================================

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner, function_tool, RunContextWrapper, ItemHelpers

=== Run starting ===
Agent updated: Joker


In [ ]:
ted_agent = Agent(
    name="Joker",
    model="gpt-4.1-nano",
    instructions="You a helpful assistant."
)

result = Runner.run_streamed(ted_agent, input="Please tell me 5 jokes.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)


Sure! Here are five jokes for you:

1. Why don’t skeletons fight each other?  
Because they don’t have the guts!

2. Why did the scarecrow win an award?  
Because he was outstanding in his field!

3. What do you call fake spaghetti?  
An impasta!

4. Why did the math book look sad?  
Because it had too many problems.

5. Why can't you trust atoms?  
Because they make up everything!  

Hope these made you smile!

In [13]:
import random

@function_tool
def how_many_jokes() -> int:
    # print("How many jokes do you want to hear?")
    return random.randint(1, 10)

    
ted_agent = Agent(
    name="Joker",
    model="gpt-4.1-nano",
    instructions="First call the 'how_many_jokes', then tell that many jokes.",
    tools=[how_many_jokes],
)
result = Runner.run_streamed(ted_agent, input="Hello")

print("=== Run starting ===")

async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print(" === Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool was called
-- Tool output: 9
-- Tool output: 7
-- Message output:
 I'll tell you 9 jokes! Here they come:

1. Why don't scientists trust atoms? Because they make up everything!
2. Why did the scarecrow win an award? Because he was outstanding in his field!
3. What do you call fake spaghetti? An impasta!
4. Why do bees have sticky hair? Because they use honeycombs!
5. What do you call a pile of cats? A meowtain!
6. Why did the bicycle fall over? Because it was two-tired!
7. What do you call cheese that isn't yours? Nacho cheese!
8. Why did the tomato turn red? Because it saw the salad dressing!
9. What did one ocean say to the other ocean? Nothing, they just waved!

Would you like to hear more jokes or anything else?
 === Run complete ===
